<a href="https://colab.research.google.com/github/Bonginkosi-Khoza/test_repo/blob/main/Mentec_milestone_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
import os

# Creating folder if it doesn’t exist
os.makedirs("stella_project", exist_ok=True)

# Changing working directory to this folder
os.chdir("stella_project")

# Checking: print current directory and its contents
print("Current working directory:", os.getcwd())
print("Files:", os.listdir())


Current working directory: /content/stella_project/stella_project
Files: []


In [24]:
#The configuration file holds global settings: secret key, database path, etc

with open("config.py", "w") as f:
    f.write("""import os

class Config:
    # Secret key: secures sessions and cookies
    SECRET_KEY = os.environ.get("SECRET_KEY", "dev-key")

    # Database: SQLite file stored as stella.db
    SQLALCHEMY_DATABASE_URI = os.environ.get("DATABASE_URL", "sqlite:///stella.db")

    # Disable modification tracking to save resources
    SQLALCHEMY_TRACK_MODIFICATIONS = False
""")

# Checking file content
with open("config.py") as f:
    print(f.read())


import os

class Config:
    # Secret key: secures sessions and cookies
    SECRET_KEY = os.environ.get("SECRET_KEY", "dev-key")

    # Database: SQLite file stored as stella.db
    SQLALCHEMY_DATABASE_URI = os.environ.get("DATABASE_URL", "sqlite:///stella.db")

    # Disable modification tracking to save resources
    SQLALCHEMY_TRACK_MODIFICATIONS = False



In [25]:


with open("app.py", "w") as f:
    f.write("""import os, io, csv, random, json
from datetime import datetime
from flask import Flask, render_template, request, redirect, url_for, flash, send_file
from flask_sqlalchemy import SQLAlchemy
from flask_login import LoginManager, UserMixin, login_user, login_required, logout_user, current_user
from werkzeug.security import generate_password_hash, check_password_hash
import matplotlib
matplotlib.use("Agg")  # Non-GUI backend for Colab
import matplotlib.pyplot as plt

from config import Config

# Create Flask app and load settings
app = Flask(__name__, template_folder="templates")
app.config.from_object(Config)

# Database setup
db = SQLAlchemy(app)

# Login manager setup
login_manager = LoginManager(app)
login_manager.login_view = "login"
""")

# Checking first 15 lines
with open("app.py") as f:
    for i, line in enumerate(f.readlines()[:15], 1):
        print(f"{i:02d}: {line}", end="")


01: import os, io, csv, random, json
02: from datetime import datetime
03: from flask import Flask, render_template, request, redirect, url_for, flash, send_file
04: from flask_sqlalchemy import SQLAlchemy
05: from flask_login import LoginManager, UserMixin, login_user, login_required, logout_user, current_user
06: from werkzeug.security import generate_password_hash, check_password_hash
07: import matplotlib
08: matplotlib.use("Agg")  # Non-GUI backend for Colab
09: import matplotlib.pyplot as plt
10: 
11: from config import Config
12: 
13: # Create Flask app and load settings
14: app = Flask(__name__, template_folder="templates")
15: app.config.from_object(Config)


In [26]:
# Nowdefining the database tables

with open("app.py", "a") as f:
    f.write("""
# ------------------- Models -------------------
class User(UserMixin, db.Model):
    id = db.Column(db.Integer, primary_key=True)
    username = db.Column(db.String(80), unique=True, nullable=False)
    email = db.Column(db.String(120), unique=True, nullable=False)
    password_hash = db.Column(db.String(255), nullable=False)

    def set_password(self, password):
        self.password_hash = generate_password_hash(password)
    def check_password(self, password):
        return check_password_hash(self.password_hash, password)

class Task(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    title = db.Column(db.String(200), nullable=False)
    is_done = db.Column(db.Boolean, default=False)
    user_id = db.Column(db.Integer, db.ForeignKey("user.id"), nullable=False)

class Expense(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    category = db.Column(db.String(100), nullable=False)
    amount = db.Column(db.Float, nullable=False)
    user_id = db.Column(db.Integer, db.ForeignKey("user.id"), nullable=False)
    created_at = db.Column(db.DateTime, default=datetime.utcnow)

@login_manager.user_loader
def load_user(user_id):
    return User.query.get(int(user_id))
""")

# Checking last 20 lines
with open("app.py") as f:
    for line in f.readlines()[-20:]:
        print(line, end="")


        self.password_hash = generate_password_hash(password)
    def check_password(self, password):
        return check_password_hash(self.password_hash, password)

class Task(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    title = db.Column(db.String(200), nullable=False)
    is_done = db.Column(db.Boolean, default=False)
    user_id = db.Column(db.Integer, db.ForeignKey("user.id"), nullable=False)

class Expense(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    category = db.Column(db.String(100), nullable=False)
    amount = db.Column(db.Float, nullable=False)
    user_id = db.Column(db.Integer, db.ForeignKey("user.id"), nullable=False)
    created_at = db.Column(db.DateTime, default=datetime.utcnow)

@login_manager.user_loader
def load_user(user_id):
    return User.query.get(int(user_id))


In [27]:
'''
outes are URLs that the app responds to. These are the core ones:

/ → homepage.

/register → user signup.

/login → user login.

/logout → log out.

/dashboard → main page after login.

'''

with open("app.py", "a") as f:
    f.write("""
# ------------------- Routes -------------------
@app.route("/")
def index():
    return render_template("index.html")

@app.route("/register", methods=["GET","POST"])
def register():
    if request.method == "POST":
        u = User(username=request.form["username"], email=request.form["email"])
        u.set_password(request.form["password"])
        db.session.add(u); db.session.commit()
        flash("Registered! Please log in.")
        return redirect(url_for("login"))
    return render_template("auth/register.html")

@app.route("/login", methods=["GET","POST"])
def login():
    if request.method == "POST":
        user = User.query.filter_by(username=request.form["username"]).first()
        if user and user.check_password(request.form["password"]):
            login_user(user)
            return redirect(url_for("dashboard"))
        flash("Invalid credentials.")
    return render_template("auth/login.html")

@app.route("/logout")
@login_required
def logout():
    logout_user()
    return redirect(url_for("index"))

@app.route("/dashboard")
@login_required
def dashboard():
    return render_template("dashboard.html")
""")

# Check last 20 lines
with open("app.py") as f:
    for line in f.readlines()[-20:]:
        print(line, end="")


@app.route("/login", methods=["GET","POST"])
def login():
    if request.method == "POST":
        user = User.query.filter_by(username=request.form["username"]).first()
        if user and user.check_password(request.form["password"]):
            login_user(user)
            return redirect(url_for("dashboard"))
        flash("Invalid credentials.")
    return render_template("auth/login.html")

@app.route("/logout")
@login_required
def logout():
    logout_user()
    return redirect(url_for("index"))

@app.route("/dashboard")
@login_required
def dashboard():
    return render_template("dashboard.html")


In [28]:
# viewing tasks and adding tasks, marking task done or undone

with open("app.py", "a") as f:
    f.write("""

@app.route("/tasks", methods=["GET","POST"])
@login_required
def tasks():
    if request.method == "POST":
        t = Task(title=request.form["title"], user_id=current_user.id)
        db.session.add(t); db.session.commit()
        return redirect(url_for("tasks"))
    items = Task.query.filter_by(user_id=current_user.id).all()
    return render_template("tasks.html", tasks=items)

@app.route("/tasks/<int:tid>/toggle")
@login_required
def toggle_task(tid):
    t = Task.query.get_or_404(tid)
    t.is_done = not t.is_done
    db.session.commit()
    return redirect(url_for("tasks"))
""")

#Checking last 15 lines
with open("app.py") as f:
    for line in f.readlines()[-15:]:
        print(line, end="")


def tasks():
    if request.method == "POST":
        t = Task(title=request.form["title"], user_id=current_user.id)
        db.session.add(t); db.session.commit()
        return redirect(url_for("tasks"))
    items = Task.query.filter_by(user_id=current_user.id).all()
    return render_template("tasks.html", tasks=items)

@app.route("/tasks/<int:tid>/toggle")
@login_required
def toggle_task(tid):
    t = Task.query.get_or_404(tid)
    t.is_done = not t.is_done
    db.session.commit()
    return redirect(url_for("tasks"))


In [29]:
# adding expense tracker routes for adding expenses, generating a pie chart of expenses and downloading csv of expenses

with open("app.py", "a") as f:
    f.write("""

@app.route("/expenses", methods=["GET","POST"])
@login_required
def expenses():
    if request.method == "POST":
        e = Expense(category=request.form["category"], amount=float(request.form["amount"]), user_id=current_user.id)
        db.session.add(e); db.session.commit()
        return redirect(url_for("expenses"))
    rows = Expense.query.filter_by(user_id=current_user.id).all()
    totals = {}
    for r in rows:
        totals[r.category] = totals.get(r.category, 0) + r.amount
    if totals:
        plt.figure()
        plt.pie(list(totals.values()), labels=list(totals.keys()), autopct="%1.1f%%")
        plt.savefig("static/expenses.png")
        plt.close()
    return render_template("expenses.html", rows=rows, totals=totals)

@app.route("/expenses/export")
@login_required
def export_expenses():
    rows = Expense.query.filter_by(user_id=current_user.id).all()
    import io, csv
    output = io.StringIO()
    writer = csv.writer(output)
    writer.writerow(["Category","Amount","Date"])
    for r in rows:
        writer.writerow([r.category, r.amount, r.created_at])
    mem = io.BytesIO(output.getvalue().encode())
    mem.seek(0)
    return send_file(mem, as_attachment=True, download_name="expenses.csv")
""")

#Checking last 25 lines
with open("app.py") as f:
    for line in f.readlines()[-25:]:
        print(line, end="")


        return redirect(url_for("expenses"))
    rows = Expense.query.filter_by(user_id=current_user.id).all()
    totals = {}
    for r in rows:
        totals[r.category] = totals.get(r.category, 0) + r.amount
    if totals:
        plt.figure()
        plt.pie(list(totals.values()), labels=list(totals.keys()), autopct="%1.1f%%")
        plt.savefig("static/expenses.png")
        plt.close()
    return render_template("expenses.html", rows=rows, totals=totals)

@app.route("/expenses/export")
@login_required
def export_expenses():
    rows = Expense.query.filter_by(user_id=current_user.id).all()
    import io, csv
    output = io.StringIO()
    writer = csv.writer(output)
    writer.writerow(["Category","Amount","Date"])
    for r in rows:
        writer.writerow([r.category, r.amount, r.created_at])
    mem = io.BytesIO(output.getvalue().encode())
    mem.seek(0)
    return send_file(mem, as_attachment=True, download_name="expenses.csv")


In [30]:
# adding a CLI command to db
with open("app.py", "a") as f:
    f.write("""

@app.cli.command("init-db")
def init_db():
    db.create_all()
    print("DB initialized.")
""")

# Checkingg last 10 lines
with open("app.py") as f:
    for line in f.readlines()[-10:]:
        print(line, end="")


        writer.writerow([r.category, r.amount, r.created_at])
    mem = io.BytesIO(output.getvalue().encode())
    mem.seek(0)
    return send_file(mem, as_attachment=True, download_name="expenses.csv")


@app.cli.command("init-db")
def init_db():
    db.create_all()
    print("DB initialized.")


In [31]:
# we use flak_ngrok
with open("app.py", "a") as f:
    f.write("""

# ------------------- Colab-specific Run -------------------
if __name__ == "__main__":
    from flask_ngrok import run_with_ngrok
    run_with_ngrok(app)   # start ngrok tunnel
    with app.app_context():
        db.create_all()   # ensure DB tables exist
    app.run()
""")

#  Checking last 15 lines
with open("app.py") as f:
    for line in f.readlines()[-15:]:
        print(line, end="")




@app.cli.command("init-db")
def init_db():
    db.create_all()
    print("DB initialized.")


# ------------------- Colab-specific Run -------------------
if __name__ == "__main__":
    from flask_ngrok import run_with_ngrok
    run_with_ngrok(app)   # start ngrok tunnel
    with app.app_context():
        db.create_all()   # ensure DB tables exist
    app.run()


In [32]:
# these templates will be used for HTML and static / for images(charts))
# Creating folders for HTML templates and static files
os.makedirs("templates/auth", exist_ok=True)
os.makedirs("static", exist_ok=True)

# Checking folder structure
for root, dirs, files in os.walk(".", topdown=True):
    print(root, "->", dirs, files)



. -> ['templates', 'static'] ['config.py', 'app.py']
./templates -> ['auth'] []
./templates/auth -> [] []
./static -> [] []


In [33]:
#(requirements.txt) this one will keep track of needed libraries
with open("requirements.txt", "w") as f:
    f.write("""flask
flask_sqlalchemy
flask_login
flask_ngrok
matplotlib
werkzeug
""")

# Checking file
with open("requirements.txt") as f:
    print(f.read())


flask
flask_sqlalchemy
flask_login
flask_ngrok
matplotlib
werkzeug



# Now we havr the backend(app.py) and DB setup, so the app experts HTML files inside templates/ and images inside static


In [34]:
# creating index.html(homepage) the landing page
with open("templates/index.html", "w") as f:
    f.write("""<!DOCTYPE html>
<html>
<head>
    <title>Stella Project</title>
</head>
<body>
    <h1>Welcome to Stella Project 🌟</h1>
    <p>
        <a href="{{ url_for('register') }}">Register</a> |
        <a href="{{ url_for('login') }}">Login</a> |
        {% if current_user.is_authenticated %}
            <a href="{{ url_for('dashboard') }}">Dashboard</a>
        {% endif %}
    </p>
</body>
</html>
""")

#Checkin the file
with open("templates/index.html") as f:
    print(f.read())


<!DOCTYPE html>
<html>
<head>
    <title>Stella Project</title>
</head>
<body>
    <h1>Welcome to Stella Project 🌟</h1>
    <p>
        <a href="{{ url_for('register') }}">Register</a> |
        <a href="{{ url_for('login') }}">Login</a> |
        {% if current_user.is_authenticated %}
            <a href="{{ url_for('dashboard') }}">Dashboard</a>
        {% endif %}
    </p>
</body>
</html>



In [35]:
# creating auth/registater.html(user registration)
#  this page lets users create an account
with open("templates/auth/register.html", "w") as f:
    f.write("""<!DOCTYPE html>
<html>
<head>
    <title>Register</title>
</head>
<body>
    <h2>Create Account</h2>
    <form method="POST">
        Username: <input type="text" name="username"><br>
        Email: <input type="email" name="email"><br>
        Password: <input type="password" name="password"><br>
        <button type="submit">Register</button>
    </form>
    <p>Already registered? <a href="{{ url_for('login') }}">Login</a></p>
</body>
</html>
""")

# Checkani file
with open("templates/auth/register.html") as f:
    print(f.read())


<!DOCTYPE html>
<html>
<head>
    <title>Register</title>
</head>
<body>
    <h2>Create Account</h2>
    <form method="POST">
        Username: <input type="text" name="username"><br>
        Email: <input type="email" name="email"><br>
        Password: <input type="password" name="password"><br>
        <button type="submit">Register</button>
    </form>
    <p>Already registered? <a href="{{ url_for('login') }}">Login</a></p>
</body>
</html>



In [36]:
# creating auth/log in html(user login)
with open("templates/auth/login.html", "w") as f:
    f.write("""<!DOCTYPE html>
<html>
<head>
    <title>Login</title>
</head>
<body>
    <h2>Login</h2>
    <form method="POST">
        Username: <input type="text" name="username"><br>
        Password: <input type="password" name="password"><br>
        <button type="submit">Login</button>
    </form>
    <p>No account? <a href="{{ url_for('register') }}">Register here</a></p>
</body>
</html>
""")

# Check file
with open("templates/auth/login.html") as f:
    print(f.read())


<!DOCTYPE html>
<html>
<head>
    <title>Login</title>
</head>
<body>
    <h2>Login</h2>
    <form method="POST">
        Username: <input type="text" name="username"><br>
        Password: <input type="password" name="password"><br>
        <button type="submit">Login</button>
    </form>
    <p>No account? <a href="{{ url_for('register') }}">Register here</a></p>
</body>
</html>



In [37]:
# creating dashboaurd.htm...this is a central page after log in. from here the users can access tasks and expenses

with open("templates/dashboard.html", "w") as f:
    f.write("""<!DOCTYPE html>
<html>
<head>
    <title>Dashboard</title>
</head>
<body>
    <h1>Hello, {{ current_user.username }} 👋</h1>
    <p>
        <a href="{{ url_for('tasks') }}">My Tasks</a> |
        <a href="{{ url_for('expenses') }}">My Expenses</a> |
        <a href="{{ url_for('logout') }}">Logout</a>
    </p>
</body>
</html>
""")

# Check file
with open("templates/dashboard.html") as f:
    print(f.read())


<!DOCTYPE html>
<html>
<head>
    <title>Dashboard</title>
</head>
<body>
    <h1>Hello, {{ current_user.username }} 👋</h1>
    <p>
        <a href="{{ url_for('tasks') }}">My Tasks</a> |
        <a href="{{ url_for('expenses') }}">My Expenses</a> |
        <a href="{{ url_for('logout') }}">Logout</a>
    </p>
</body>
</html>



In [38]:
# create tasks.html...to show task list, allowing adding tasks, toggling done/undone
with open("templates/tasks.html", "w") as f:
    f.write("""<!DOCTYPE html>
<html>
<head>
    <title>Tasks</title>
</head>
<body>
    <h2>My To-Do List</h2>
    <form method="POST">
        <input type="text" name="title" placeholder="New Task">
        <button type="submit">Add</button>
    </form>
    <ul>
        {% for t in tasks %}
            <li>
                {{ "✅" if t.is_done else "❌" }}
                {{ t.title }}
                <a href="{{ url_for('toggle_task', tid=t.id) }}">[Toggle]</a>
            </li>
        {% endfor %}
    </ul>
    <p><a href="{{ url_for('dashboard') }}">Back</a></p>
</body>
</html>
""")

# 🔎 Check file
with open("templates/tasks.html") as f:
    print(f.read())


<!DOCTYPE html>
<html>
<head>
    <title>Tasks</title>
</head>
<body>
    <h2>My To-Do List</h2>
    <form method="POST">
        <input type="text" name="title" placeholder="New Task">
        <button type="submit">Add</button>
    </form>
    <ul>
        {% for t in tasks %}
            <li>
                {{ "✅" if t.is_done else "❌" }}
                {{ t.title }}
                <a href="{{ url_for('toggle_task', tid=t.id) }}">[Toggle]</a>
            </li>
        {% endfor %}
    </ul>
    <p><a href="{{ url_for('dashboard') }}">Back</a></p>
</body>
</html>



In [39]:
# creating expenses.html...to show expenses list, allow ading, display a oie chart, csv export

with open("templates/expenses.html", "w") as f:
    f.write("""<!DOCTYPE html>
<html>
<head>
    <title>Expenses</title>
</head>
<body>
    <h2>Expense Tracker</h2>
    <form method="POST">
        Category: <input type="text" name="category">
        Amount: <input type="number" step="0.01" name="amount">
        <button type="submit">Add</button>
    </form>

    <h3>All Expenses</h3>
    <ul>
        {% for r in rows %}
            <li>{{ r.category }} - ${{ r.amount }} ({{ r.created_at.strftime('%Y-%m-%d') }})</li>
        {% endfor %}
    </ul>

    {% if totals %}
        <h3>Category Breakdown</h3>
        <img src="{{ url_for('static', filename='expenses.png') }}" width="300">
    {% endif %}

    <p><a href="{{ url_for('export_expenses') }}">Download CSV</a></p>
    <p><a href="{{ url_for('dashboard') }}">Back</a></p>
</body>
</html>
""")

#Check file
with open("templates/expenses.html") as f:
    print(f.read())


<!DOCTYPE html>
<html>
<head>
    <title>Expenses</title>
</head>
<body>
    <h2>Expense Tracker</h2>
    <form method="POST">
        Category: <input type="text" name="category">
        Amount: <input type="number" step="0.01" name="amount">
        <button type="submit">Add</button>
    </form>

    <h3>All Expenses</h3>
    <ul>
        {% for r in rows %}
            <li>{{ r.category }} - ${{ r.amount }} ({{ r.created_at.strftime('%Y-%m-%d') }})</li>
        {% endfor %}
    </ul>

    {% if totals %}
        <h3>Category Breakdown</h3>
        <img src="{{ url_for('static', filename='expenses.png') }}" width="300">
    {% endif %}

    <p><a href="{{ url_for('export_expenses') }}">Download CSV</a></p>
    <p><a href="{{ url_for('dashboard') }}">Back</a></p>
</body>
</html>



In [40]:
# VERIFYING ALL TEMPLATES EXIST
#confirming al the templates files are iin place
for root, dirs, files in os.walk("templates", topdown=True):
    print(root, "->", files)


templates -> ['expenses.html', 'dashboard.html', 'index.html', 'tasks.html']
templates/auth -> ['login.html', 'register.html']


# Frontend and backend are ready, we install dependencies and run the flask app inside colab


In [41]:
# installing dependencies, the requirements listes in requirements.text, now im installing them inside colab using pip
!pip install -r requirements.txt


In [42]:
# initialing the database, with tthe tables users, tasks, expenses with a python snippet
from app import db, app

# Creating DB tables
with app.app_context():
    db.create_all()

print(" Database initialized (stella.db created).")


 Database initialized (stella.db created).


In [ ]:
# runnning the flask app with ngrok
!pkill gunicorn || echo "No gunicorn process running"  # clean any previous server

from app import app
from flask_ngrok import run_with_ngrok

# Starting ngrok tunnel
run_with_ngrok(app)

# Running the app
app.run()


No gunicorn process running
 * Serving Flask app 'app'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
Exception in thread Thread-10:
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/urllib3/connection.py", line 198, in _new_conn
    sock = connection.create_connection(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/urllib3/util/connection.py", line 85, in create_connection
    raise err
  File "/usr/local/lib/python3.12/dist-packages/urllib3/util/connection.py", line 73, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [Errno 111] Connection refused

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/urllib3/connectionpool.py", line 787, in urlopen
    response = self._make_request(
           

# Testing the app manually
## we will test by opening the ngrok link